## 读取数据

In [88]:
# 首先 import 必要的模块
import pandas as pd #数据分析
import numpy as np #科学计算
from pandas import Series,DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
#打开文件
# path to where the data lies
dpath = './input/'
train = pd.read_csv(dpath +'train_data_org.csv',index_col='用户编码')
y_train=train.pop('信用分')
X_train=train.drop(['data_type'], axis=1)

In [89]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
def score(y_pred, y_true):
    return 1/(mean_absolute_error(y_pred, y_true)+1)
My_Score = make_scorer(score, greater_is_better=True)

## 模型选择

In [90]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
import warnings
def ignore_warn(*args, **kwargs):
    pass

In [91]:
#Validation function
def score_cv(model):
    score=cross_val_score(model, X_train.values, y_train, scoring=My_Score, cv = 5)
    return(score)

# 基模型

In [52]:
# Ridge Regression
from sklearn.linear_model import Ridge
Ridge= make_pipeline( Ridge(alpha =9.5))

In [53]:
# LASSO Regression 
from sklearn.linear_model import Lasso
lasso = make_pipeline( Lasso(alpha =0.0001))

In [54]:
# Elastic Net Regression
from sklearn.linear_model import ElasticNet
ENet = make_pipeline(ElasticNet(alpha=0.000001, l1_ratio=0.5,random_state=3))

In [55]:
# KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
KNN = make_pipeline( KNeighborsRegressor(n_neighbors=8,weights='distance',n_jobs=4))

In [56]:
# Bayesian Ridge Regression
from sklearn.linear_model import BayesianRidge
Bayes = make_pipeline(BayesianRidge(alpha_1=1e-06,lambda_1=1e-06,alpha_2=1e-06,lambda_2=1e-06))

In [57]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
# Bayesian Ridge Regression
BagTree= make_pipeline( BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=0),n_estimators=200,n_jobs=4))

In [58]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
RandomForest = make_pipeline(RandomForestRegressor(max_depth=25, random_state=0,n_estimators=100,n_jobs=4,warm_start=True,verbose=3))

In [59]:
#Gradient Boosting Regression
from sklearn.ensemble import GradientBoostingRegressor
GBoost = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='lad', random_state =5)

In [60]:
# XGBRegressor
import xgboost as xgb
other_params = {'learning_rate': 0.01, 'n_estimators': 2000, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                'n_jobs':4,'subsample': 0.93, 'colsample_bytree': 0.7, 'gamma': 0.000000001, 'reg_alpha': 0.000001, 'reg_lambda': 1}
Xgb = xgb.XGBRegressor(**other_params)

In [61]:
# LGBMRegressor
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
Lgbm1 = lgb.LGBMRegressor(objective='regression_l1',num_leaves=30,bagging_fraction=0.6,reg_alpha=0.001, reg_lambda=0.001,
                              learning_rate=0.01, n_estimators=4000, max_depth=6,feature_fraction=0.5,
                              metric='mae', bagging_freq = 5,  min_child_samples=17,min_child_weight=0.00001)

In [62]:
# LGBMRegressor
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
Lgbm2 = lgb.LGBMRegressor(objective='regression_l2',num_leaves=30,bagging_fraction=0.6,reg_alpha=0.001, reg_lambda=0.001,
                              learning_rate=0.01, n_estimators=4000, max_depth=6,feature_fraction=0.5,
                              metric='mae', bagging_freq = 5,  min_child_samples=17,min_child_weight=0.00001)

In [63]:
# CatBoostRegressor
from catboost import CatBoostRegressor
Catb= CatBoostRegressor(iterations=2500, depth=3, learning_rate=0.15, loss_function='RMSE')

# 各个基模型得分

In [25]:
score_Ridge = score_cv(Ridge)
print("\nRidge score: {:.7f} ({:.4f})\n".format(score_Ridge.mean(), score_Ridge.std()))


Ridge score: 0.0577158 (0.0003)



In [26]:
score_lasso = score_cv(lasso)
print("\nLasso score: {:.7f} ({:.4f})\n".format(score_lasso.mean(), score_lasso.std()))

D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very 


Lasso score: 0.0576907 (0.0003)



D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [23]:
score_ENet = score_cv(ENet)
print("ElasticNet score: {:.7f} ({:.4f})\n".format(score_ENet.mean(),score_ENet.std()))

D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very 

ElasticNet score: 0.0600569 (0.0004)



D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [24]:
score_KNN = score_cv(KNN)
print("Kernel Ridge score: {:.7f} ({:.4f})\n".format(score_KNN.mean(), score_KNN.std()))

Kernel Ridge score: 0.0426380 (0.0003)



In [25]:
score_Bayes = score_cv(Bayes)
print("Kernel Ridge score: {:.7f} ({:.4f})\n".format(score_Bayes.mean(),score_Bayes.std()))

Kernel Ridge score: 0.0600620 (0.0004)



In [26]:
score_BagTree = score_cv(BagTree)
print("Kernel Ridge score: {:.7f} ({:.4f})\n".format(score_BagTree.mean(),score_BagTree .std()))

Kernel Ridge score: 0.0608932 (0.0005)



In [27]:
score_RandomForest = score_cv(RandomForest)
print("Kernel Ridge score: {:.7f} ({:.4f})\n".format(score_RandomForest.mean(),score_RandomForest.std()))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   35.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   37.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   37.5s
[Parallel(n_jobs=4)]: Done 100

Kernel Ridge score: 0.0607342 (0.0005)



[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


In [28]:
score_GBoost = score_cv(GBoost)
print("Gradient Boosting score: {:.7f} ({:.4f})\n".format(score_GBoost.mean(),score_GBoost.std()))

Gradient Boosting score: 0.0622818 (0.0004)



In [29]:
score_Xgb = score_cv(Xgb)
print("Xgboost score: {:.7f} ({:.4f})\n".format(score_Xgb.mean(), score_Xgb.std()))

Xgboost score: 0.0629298 (0.0005)



In [28]:
score_Lgbm1= score_cv(Lgbm1)
print("LGBM score: {:.7f} ({:.4f})\n" .format(score_Lgbm1.mean(),score_Lgbm1.std()))

LGBM score: 0.0638346 (0.0005)



In [29]:
score_Lgbm2= score_cv(Lgbm2)
print("LGBM score: {:.7f} ({:.4f})\n" .format(score_Lgbm2.mean(),score_Lgbm2.std()))

LGBM score: 0.0636635 (0.0005)



In [30]:
score_Catb= score_cv(Catb)
print("LGBM score: {:.7f} ({:.4f})\n" .format(score_Catb.mean(),score_Catb.std()))

0:	learn: 527.1478980	total: 85.4ms	remaining: 3m 33s
1:	learn: 448.6536514	total: 131ms	remaining: 2m 43s
2:	learn: 382.0043071	total: 173ms	remaining: 2m 24s
3:	learn: 325.4698802	total: 210ms	remaining: 2m 11s
4:	learn: 277.3527910	total: 248ms	remaining: 2m 3s
5:	learn: 236.6842764	total: 287ms	remaining: 1m 59s
6:	learn: 202.2714899	total: 320ms	remaining: 1m 54s
7:	learn: 173.0198352	total: 358ms	remaining: 1m 51s
8:	learn: 148.0471485	total: 393ms	remaining: 1m 48s
9:	learn: 127.2993605	total: 427ms	remaining: 1m 46s
10:	learn: 109.8569892	total: 464ms	remaining: 1m 44s
11:	learn: 94.8836093	total: 501ms	remaining: 1m 43s
12:	learn: 82.1691143	total: 547ms	remaining: 1m 44s
13:	learn: 71.3306598	total: 595ms	remaining: 1m 45s
14:	learn: 62.1425728	total: 633ms	remaining: 1m 44s
15:	learn: 54.4231777	total: 669ms	remaining: 1m 43s
16:	learn: 48.0791376	total: 702ms	remaining: 1m 42s
17:	learn: 43.0778544	total: 740ms	remaining: 1m 42s
18:	learn: 38.7929816	total: 777ms	remaining:

158:	learn: 19.5816976	total: 5.72s	remaining: 1m 24s
159:	learn: 19.5815198	total: 5.75s	remaining: 1m 24s
160:	learn: 19.5751974	total: 5.78s	remaining: 1m 24s
161:	learn: 19.5707085	total: 5.82s	remaining: 1m 23s
162:	learn: 19.5687293	total: 5.85s	remaining: 1m 23s
163:	learn: 19.5669361	total: 5.88s	remaining: 1m 23s
164:	learn: 19.5667021	total: 5.92s	remaining: 1m 23s
165:	learn: 19.5579800	total: 5.96s	remaining: 1m 23s
166:	learn: 19.5570950	total: 5.99s	remaining: 1m 23s
167:	learn: 19.5552127	total: 6.02s	remaining: 1m 23s
168:	learn: 19.5469364	total: 6.06s	remaining: 1m 23s
169:	learn: 19.5433301	total: 6.09s	remaining: 1m 23s
170:	learn: 19.5385832	total: 6.13s	remaining: 1m 23s
171:	learn: 19.5361714	total: 6.16s	remaining: 1m 23s
172:	learn: 19.5271887	total: 6.2s	remaining: 1m 23s
173:	learn: 19.5249192	total: 6.23s	remaining: 1m 23s
174:	learn: 19.5229053	total: 6.26s	remaining: 1m 23s
175:	learn: 19.5201617	total: 6.3s	remaining: 1m 23s
176:	learn: 19.5201360	total: 

315:	learn: 19.2474476	total: 11.1s	remaining: 1m 16s
316:	learn: 19.2474462	total: 11.2s	remaining: 1m 16s
317:	learn: 19.2474427	total: 11.2s	remaining: 1m 16s
318:	learn: 19.2459969	total: 11.2s	remaining: 1m 16s
319:	learn: 19.2450585	total: 11.3s	remaining: 1m 16s
320:	learn: 19.2430878	total: 11.3s	remaining: 1m 16s
321:	learn: 19.2430845	total: 11.3s	remaining: 1m 16s
322:	learn: 19.2412577	total: 11.4s	remaining: 1m 16s
323:	learn: 19.2387992	total: 11.4s	remaining: 1m 16s
324:	learn: 19.2355432	total: 11.4s	remaining: 1m 16s
325:	learn: 19.2327562	total: 11.5s	remaining: 1m 16s
326:	learn: 19.2327538	total: 11.5s	remaining: 1m 16s
327:	learn: 19.2327515	total: 11.5s	remaining: 1m 16s
328:	learn: 19.2199905	total: 11.6s	remaining: 1m 16s
329:	learn: 19.2199853	total: 11.6s	remaining: 1m 16s
330:	learn: 19.2199832	total: 11.6s	remaining: 1m 16s
331:	learn: 19.2181041	total: 11.7s	remaining: 1m 16s
332:	learn: 19.2179755	total: 11.7s	remaining: 1m 16s
333:	learn: 19.2179755	total

468:	learn: 19.0889722	total: 17.1s	remaining: 1m 13s
469:	learn: 19.0889644	total: 17.1s	remaining: 1m 13s
470:	learn: 19.0882088	total: 17.1s	remaining: 1m 13s
471:	learn: 19.0880713	total: 17.2s	remaining: 1m 13s
472:	learn: 19.0868872	total: 17.2s	remaining: 1m 13s
473:	learn: 19.0862105	total: 17.3s	remaining: 1m 13s
474:	learn: 19.0852121	total: 17.3s	remaining: 1m 13s
475:	learn: 19.0837605	total: 17.3s	remaining: 1m 13s
476:	learn: 19.0812267	total: 17.4s	remaining: 1m 13s
477:	learn: 19.0812266	total: 17.4s	remaining: 1m 13s
478:	learn: 19.0796183	total: 17.4s	remaining: 1m 13s
479:	learn: 19.0774667	total: 17.5s	remaining: 1m 13s
480:	learn: 19.0773234	total: 17.5s	remaining: 1m 13s
481:	learn: 19.0770034	total: 17.5s	remaining: 1m 13s
482:	learn: 19.0768360	total: 17.6s	remaining: 1m 13s
483:	learn: 19.0768360	total: 17.6s	remaining: 1m 13s
484:	learn: 19.0768360	total: 17.6s	remaining: 1m 13s
485:	learn: 19.0768357	total: 17.7s	remaining: 1m 13s
486:	learn: 19.0768355	total

625:	learn: 18.9678627	total: 22.6s	remaining: 1m 7s
626:	learn: 18.9672741	total: 22.6s	remaining: 1m 7s
627:	learn: 18.9661721	total: 22.7s	remaining: 1m 7s
628:	learn: 18.9658837	total: 22.7s	remaining: 1m 7s
629:	learn: 18.9642402	total: 22.7s	remaining: 1m 7s
630:	learn: 18.9630707	total: 22.8s	remaining: 1m 7s
631:	learn: 18.9618284	total: 22.8s	remaining: 1m 7s
632:	learn: 18.9613351	total: 22.8s	remaining: 1m 7s
633:	learn: 18.9610056	total: 22.9s	remaining: 1m 7s
634:	learn: 18.9598068	total: 22.9s	remaining: 1m 7s
635:	learn: 18.9593824	total: 22.9s	remaining: 1m 7s
636:	learn: 18.9593822	total: 23s	remaining: 1m 7s
637:	learn: 18.9584673	total: 23s	remaining: 1m 7s
638:	learn: 18.9566136	total: 23s	remaining: 1m 7s
639:	learn: 18.9561806	total: 23.1s	remaining: 1m 7s
640:	learn: 18.9555056	total: 23.1s	remaining: 1m 7s
641:	learn: 18.9551416	total: 23.1s	remaining: 1m 6s
642:	learn: 18.9550988	total: 23.2s	remaining: 1m 6s
643:	learn: 18.9550985	total: 23.2s	remaining: 1m 6s

...

1844:	learn: 18.5929972	total: 1m 17s	remaining: 27.5s
1845:	learn: 18.5927799	total: 1m 17s	remaining: 27.4s
1846:	learn: 18.5927280	total: 1m 17s	remaining: 27.4s
1847:	learn: 18.5927280	total: 1m 17s	remaining: 27.3s
1848:	learn: 18.5921336	total: 1m 17s	remaining: 27.3s
1849:	learn: 18.5914662	total: 1m 17s	remaining: 27.3s
1850:	learn: 18.5914034	total: 1m 17s	remaining: 27.2s
1851:	learn: 18.5913993	total: 1m 17s	remaining: 27.2s
1852:	learn: 18.5913982	total: 1m 17s	remaining: 27.1s
1853:	learn: 18.5913974	total: 1m 17s	remaining: 27.1s
1854:	learn: 18.5913974	total: 1m 17s	remaining: 27s
1855:	learn: 18.5913974	total: 1m 17s	remaining: 27s
1856:	learn: 18.5912141	total: 1m 17s	remaining: 27s
1857:	learn: 18.5910859	total: 1m 17s	remaining: 26.9s
1858:	learn: 18.5910859	total: 1m 17s	remaining: 26.9s
1859:	learn: 18.5910447	total: 1m 17s	remaining: 26.8s
1860:	learn: 18.5908172	total: 1m 18s	remaining: 26.8s
1861:	learn: 18.5906289	total: 1m 18s	remaining: 26.8s
1862:	learn: 18.

1996:	learn: 18.5697100	total: 1m 23s	remaining: 21s
1997:	learn: 18.5697072	total: 1m 23s	remaining: 21s
1998:	learn: 18.5697067	total: 1m 23s	remaining: 20.9s
1999:	learn: 18.5697046	total: 1m 23s	remaining: 20.9s
2000:	learn: 18.5696105	total: 1m 23s	remaining: 20.8s
2001:	learn: 18.5695022	total: 1m 23s	remaining: 20.8s
2002:	learn: 18.5694877	total: 1m 23s	remaining: 20.8s
2003:	learn: 18.5694657	total: 1m 23s	remaining: 20.7s
2004:	learn: 18.5694512	total: 1m 23s	remaining: 20.7s
2005:	learn: 18.5690923	total: 1m 23s	remaining: 20.6s
2006:	learn: 18.5690585	total: 1m 23s	remaining: 20.6s
2007:	learn: 18.5690499	total: 1m 23s	remaining: 20.5s
2008:	learn: 18.5689766	total: 1m 23s	remaining: 20.5s
2009:	learn: 18.5683565	total: 1m 23s	remaining: 20.5s
2010:	learn: 18.5673800	total: 1m 23s	remaining: 20.4s
2011:	learn: 18.5673800	total: 1m 23s	remaining: 20.4s
2012:	learn: 18.5673439	total: 1m 24s	remaining: 20.3s
2013:	learn: 18.5673439	total: 1m 24s	remaining: 20.3s
2014:	learn: 1

2146:	learn: 18.5486790	total: 1m 29s	remaining: 14.7s
2147:	learn: 18.5486789	total: 1m 29s	remaining: 14.6s
2148:	learn: 18.5474594	total: 1m 29s	remaining: 14.6s
2149:	learn: 18.5473795	total: 1m 29s	remaining: 14.5s
2150:	learn: 18.5472312	total: 1m 29s	remaining: 14.5s
2151:	learn: 18.5469445	total: 1m 29s	remaining: 14.5s
2152:	learn: 18.5469444	total: 1m 29s	remaining: 14.4s
2153:	learn: 18.5469436	total: 1m 29s	remaining: 14.4s
2154:	learn: 18.5469181	total: 1m 29s	remaining: 14.3s
2155:	learn: 18.5468312	total: 1m 29s	remaining: 14.3s
2156:	learn: 18.5467685	total: 1m 29s	remaining: 14.3s
2157:	learn: 18.5455039	total: 1m 29s	remaining: 14.2s
2158:	learn: 18.5447542	total: 1m 29s	remaining: 14.2s
2159:	learn: 18.5447189	total: 1m 29s	remaining: 14.1s
2160:	learn: 18.5447171	total: 1m 29s	remaining: 14.1s
2161:	learn: 18.5447171	total: 1m 29s	remaining: 14s
2162:	learn: 18.5447170	total: 1m 29s	remaining: 14s
2163:	learn: 18.5442240	total: 1m 29s	remaining: 14s
2164:	learn: 18.

2298:	learn: 18.5238090	total: 1m 35s	remaining: 8.37s
2299:	learn: 18.5238085	total: 1m 35s	remaining: 8.32s
2300:	learn: 18.5237541	total: 1m 35s	remaining: 8.28s
2301:	learn: 18.5237509	total: 1m 35s	remaining: 8.24s
2302:	learn: 18.5237487	total: 1m 35s	remaining: 8.2s
2303:	learn: 18.5237487	total: 1m 35s	remaining: 8.16s
2304:	learn: 18.5235523	total: 1m 35s	remaining: 8.12s
2305:	learn: 18.5230420	total: 1m 35s	remaining: 8.07s
2306:	learn: 18.5230420	total: 1m 35s	remaining: 8.03s
2307:	learn: 18.5226446	total: 1m 36s	remaining: 7.99s
2308:	learn: 18.5226336	total: 1m 36s	remaining: 7.95s
2309:	learn: 18.5226205	total: 1m 36s	remaining: 7.91s
2310:	learn: 18.5226205	total: 1m 36s	remaining: 7.86s
2311:	learn: 18.5226182	total: 1m 36s	remaining: 7.82s
2312:	learn: 18.5226182	total: 1m 36s	remaining: 7.78s
2313:	learn: 18.5220894	total: 1m 36s	remaining: 7.74s
2314:	learn: 18.5220890	total: 1m 36s	remaining: 7.7s
2315:	learn: 18.5220530	total: 1m 36s	remaining: 7.66s
2316:	learn:

2449:	learn: 18.5048146	total: 1m 41s	remaining: 2.07s
2450:	learn: 18.5039674	total: 1m 41s	remaining: 2.03s
2451:	learn: 18.5035376	total: 1m 41s	remaining: 1.99s
2452:	learn: 18.5035374	total: 1m 41s	remaining: 1.95s
2453:	learn: 18.5035370	total: 1m 41s	remaining: 1.91s
2454:	learn: 18.5035365	total: 1m 41s	remaining: 1.86s
2455:	learn: 18.5035359	total: 1m 41s	remaining: 1.82s
2456:	learn: 18.5035359	total: 1m 41s	remaining: 1.78s
2457:	learn: 18.5035089	total: 1m 41s	remaining: 1.74s
2458:	learn: 18.5035089	total: 1m 41s	remaining: 1.7s
2459:	learn: 18.5035087	total: 1m 41s	remaining: 1.66s
2460:	learn: 18.5035087	total: 1m 41s	remaining: 1.61s
2461:	learn: 18.5035087	total: 1m 41s	remaining: 1.57s
2462:	learn: 18.5035087	total: 1m 42s	remaining: 1.53s
2463:	learn: 18.5035086	total: 1m 42s	remaining: 1.49s
2464:	learn: 18.5035068	total: 1m 42s	remaining: 1.45s
2465:	learn: 18.5033584	total: 1m 42s	remaining: 1.41s
2466:	learn: 18.5032078	total: 1m 42s	remaining: 1.37s
2467:	learn

## 模型融合

### 1.Average-Models

In [53]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models

    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self

    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1) 

In [54]:
#averaged_models = AveragingModels(models = ( Ridge,lasso,ENet,KNN,Bayes ))
averaged_models = AveragingModels(models = ( Ridge,BagTree,RandomForest,GBoost,Xgb,Lgbm ,Catb))
score= score_cv(averaged_models)
print("Averaged base models score: {:.7f} ({:.4f})\n" .format(score.mean(),score.std()))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   10.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   42.9s finished


0:	learn: 434.8093874	total: 29.4ms	remaining: 14.7s
1:	learn: 305.8903942	total: 59ms	remaining: 14.7s
2:	learn: 216.2615752	total: 84.5ms	remaining: 14s
3:	learn: 154.2372387	total: 108ms	remaining: 13.4s
4:	learn: 110.4315752	total: 137ms	remaining: 13.5s
5:	learn: 80.2227862	total: 162ms	remaining: 13.4s
6:	learn: 59.2569653	total: 191ms	remaining: 13.4s
7:	learn: 45.2602843	total: 217ms	remaining: 13.3s
8:	learn: 36.2271916	total: 250ms	remaining: 13.6s
9:	learn: 30.5629787	total: 297ms	remaining: 14.5s
10:	learn: 27.1178504	total: 339ms	remaining: 15.1s
11:	learn: 25.0064651	total: 377ms	remaining: 15.3s
12:	learn: 23.7756926	total: 410ms	remaining: 15.4s
13:	learn: 23.1064454	total: 467ms	remaining: 16.2s
14:	learn: 22.6004702	total: 507ms	remaining: 16.4s
15:	learn: 22.2044077	total: 547ms	remaining: 16.5s
16:	learn: 21.9270343	total: 574ms	remaining: 16.3s
17:	learn: 21.7208651	total: 601ms	remaining: 16.1s
18:	learn: 21.5364868	total: 629ms	remaining: 15.9s
19:	learn: 21.3664

160:	learn: 19.6852146	total: 5.86s	remaining: 12.3s
161:	learn: 19.6828954	total: 5.89s	remaining: 12.3s
162:	learn: 19.6743072	total: 5.92s	remaining: 12.2s
163:	learn: 19.6693073	total: 5.95s	remaining: 12.2s
164:	learn: 19.6692299	total: 5.98s	remaining: 12.1s
165:	learn: 19.6662179	total: 6.01s	remaining: 12.1s
166:	learn: 19.6536451	total: 6.04s	remaining: 12s
167:	learn: 19.6536228	total: 6.06s	remaining: 12s
168:	learn: 19.6527043	total: 6.09s	remaining: 11.9s
169:	learn: 19.6462491	total: 6.12s	remaining: 11.9s
170:	learn: 19.6413221	total: 6.15s	remaining: 11.8s
171:	learn: 19.6406902	total: 6.17s	remaining: 11.8s
172:	learn: 19.6402137	total: 6.2s	remaining: 11.7s
173:	learn: 19.6386994	total: 6.23s	remaining: 11.7s
174:	learn: 19.6377935	total: 6.26s	remaining: 11.6s
175:	learn: 19.6330606	total: 6.29s	remaining: 11.6s
176:	learn: 19.6327357	total: 6.32s	remaining: 11.5s
177:	learn: 19.6286268	total: 6.35s	remaining: 11.5s
178:	learn: 19.6285767	total: 6.37s	remaining: 11.4

321:	learn: 19.3330806	total: 10.6s	remaining: 5.84s
322:	learn: 19.3330806	total: 10.6s	remaining: 5.8s
323:	learn: 19.3330692	total: 10.6s	remaining: 5.77s
324:	learn: 19.3330606	total: 10.7s	remaining: 5.74s
325:	learn: 19.3328821	total: 10.7s	remaining: 5.7s
326:	learn: 19.3328810	total: 10.7s	remaining: 5.66s
327:	learn: 19.3325581	total: 10.7s	remaining: 5.63s
328:	learn: 19.3320967	total: 10.8s	remaining: 5.59s
329:	learn: 19.3320955	total: 10.8s	remaining: 5.57s
330:	learn: 19.3307400	total: 10.9s	remaining: 5.54s
331:	learn: 19.3282133	total: 10.9s	remaining: 5.52s
332:	learn: 19.3263842	total: 10.9s	remaining: 5.49s
333:	learn: 19.3238987	total: 11s	remaining: 5.46s
334:	learn: 19.3238729	total: 11s	remaining: 5.44s
335:	learn: 19.3238694	total: 11.1s	remaining: 5.41s
336:	learn: 19.3228902	total: 11.1s	remaining: 5.38s
337:	learn: 19.3228113	total: 11.2s	remaining: 5.35s
338:	learn: 19.3207383	total: 11.2s	remaining: 5.32s
339:	learn: 19.3203727	total: 11.2s	remaining: 5.28s

480:	learn: 19.1643531	total: 15.7s	remaining: 621ms
481:	learn: 19.1631735	total: 15.7s	remaining: 588ms
482:	learn: 19.1618502	total: 15.8s	remaining: 555ms
483:	learn: 19.1605403	total: 15.8s	remaining: 523ms
484:	learn: 19.1604099	total: 15.8s	remaining: 490ms
485:	learn: 19.1602081	total: 15.9s	remaining: 457ms
486:	learn: 19.1600971	total: 15.9s	remaining: 424ms
487:	learn: 19.1588309	total: 15.9s	remaining: 392ms
488:	learn: 19.1579469	total: 16s	remaining: 359ms
489:	learn: 19.1579344	total: 16s	remaining: 326ms
490:	learn: 19.1579301	total: 16s	remaining: 293ms
491:	learn: 19.1576740	total: 16s	remaining: 261ms
492:	learn: 19.1576738	total: 16.1s	remaining: 228ms
493:	learn: 19.1573792	total: 16.1s	remaining: 195ms
494:	learn: 19.1558964	total: 16.1s	remaining: 163ms
495:	learn: 19.1553607	total: 16.2s	remaining: 130ms
496:	learn: 19.1548858	total: 16.2s	remaining: 97.7ms
497:	learn: 19.1543307	total: 16.2s	remaining: 65.1ms
498:	learn: 19.1543097	total: 16.2s	remaining: 32.5m

...

161:	learn: 19.5942724	total: 5.66s	remaining: 11.8s
162:	learn: 19.5932723	total: 5.7s	remaining: 11.8s
163:	learn: 19.5830417	total: 5.75s	remaining: 11.8s
164:	learn: 19.5725272	total: 5.78s	remaining: 11.7s
165:	learn: 19.5711428	total: 5.81s	remaining: 11.7s
166:	learn: 19.5654173	total: 5.84s	remaining: 11.7s
167:	learn: 19.5639426	total: 5.89s	remaining: 11.6s
168:	learn: 19.5639273	total: 5.92s	remaining: 11.6s
169:	learn: 19.5639124	total: 5.95s	remaining: 11.6s
170:	learn: 19.5634373	total: 5.98s	remaining: 11.5s
171:	learn: 19.5633933	total: 6.01s	remaining: 11.5s
172:	learn: 19.5633601	total: 6.04s	remaining: 11.4s
173:	learn: 19.5633357	total: 6.09s	remaining: 11.4s
174:	learn: 19.5553572	total: 6.13s	remaining: 11.4s
175:	learn: 19.5550461	total: 6.17s	remaining: 11.4s
176:	learn: 19.5497337	total: 6.2s	remaining: 11.3s
177:	learn: 19.5468543	total: 6.23s	remaining: 11.3s
178:	learn: 19.5407895	total: 6.27s	remaining: 11.2s
179:	learn: 19.5407627	total: 6.3s	remaining: 11

318:	learn: 19.3064718	total: 11.1s	remaining: 6.27s
319:	learn: 19.3064699	total: 11.1s	remaining: 6.24s
320:	learn: 19.3052083	total: 11.1s	remaining: 6.2s
321:	learn: 19.3051061	total: 11.2s	remaining: 6.17s
322:	learn: 19.3044434	total: 11.2s	remaining: 6.13s
323:	learn: 19.3044318	total: 11.2s	remaining: 6.1s
324:	learn: 19.3044259	total: 11.3s	remaining: 6.06s
325:	learn: 19.3017187	total: 11.3s	remaining: 6.03s
326:	learn: 19.3017170	total: 11.3s	remaining: 5.99s
327:	learn: 19.3016415	total: 11.4s	remaining: 5.96s
328:	learn: 19.3016366	total: 11.4s	remaining: 5.92s
329:	learn: 19.3014750	total: 11.4s	remaining: 5.88s
330:	learn: 19.3003124	total: 11.4s	remaining: 5.84s
331:	learn: 19.3003108	total: 11.5s	remaining: 5.81s
332:	learn: 19.2988350	total: 11.5s	remaining: 5.78s
333:	learn: 19.2979927	total: 11.5s	remaining: 5.74s
334:	learn: 19.2963563	total: 11.6s	remaining: 5.7s
335:	learn: 19.2963548	total: 11.6s	remaining: 5.66s
336:	learn: 19.2924812	total: 11.6s	remaining: 5.

479:	learn: 19.1298849	total: 16.1s	remaining: 672ms
480:	learn: 19.1295006	total: 16.2s	remaining: 638ms
481:	learn: 19.1289749	total: 16.2s	remaining: 605ms
482:	learn: 19.1284465	total: 16.2s	remaining: 572ms
483:	learn: 19.1276444	total: 16.3s	remaining: 538ms
484:	learn: 19.1276194	total: 16.3s	remaining: 504ms
485:	learn: 19.1274668	total: 16.3s	remaining: 470ms
486:	learn: 19.1250761	total: 16.3s	remaining: 436ms
487:	learn: 19.1249633	total: 16.4s	remaining: 403ms
488:	learn: 19.1249474	total: 16.4s	remaining: 369ms
489:	learn: 19.1242011	total: 16.4s	remaining: 336ms
490:	learn: 19.1219908	total: 16.5s	remaining: 302ms
491:	learn: 19.1219819	total: 16.5s	remaining: 268ms
492:	learn: 19.1199522	total: 16.5s	remaining: 235ms
493:	learn: 19.1163766	total: 16.6s	remaining: 201ms
494:	learn: 19.1163766	total: 16.6s	remaining: 168ms
495:	learn: 19.1126004	total: 16.6s	remaining: 134ms
496:	learn: 19.1125803	total: 16.6s	remaining: 100ms
497:	learn: 19.1125803	total: 16.7s	remaining:

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


Averaged base models score: 0.0631491 (0.0004)



In [23]:
#averaged_models1 = AveragingModels(models = ( Ridge,lasso,ENet,KNN,Bayes ))
averaged_models = AveragingModels(models = ( Ridge,BagTree,RandomForest,GBoost,Xgb,Lgbm ,Catb))
score= score_cv(averaged_models)
print("Averaged base models score: {:.7f} ({:.4f})\n" .format(score.mean(),score.std()))

Averaged base models score: 0.0633016 (0.0004)



In [55]:
averaged_models = AveragingModels(models = ( Ridge,BagTree,RandomForest,GBoost,Xgb,Lgbm ,Catb))
averaged_models.fit(X_train, y_train)
#打开文件
# path to where the data lies
dpath = './input/'
test = pd.read_csv(dpath +'test_data_org.csv',index_col='用户编码')
X_test=test.drop(['data_type'], axis=1)
X_test_index=X_test.index
#y预测，四舍五入，取整
y_pred=np.around(averaged_models.predict(X_test)).astype(int)
#生成提交结果
user_id=pd.DataFrame(X_test_index).rename(columns = {'用户编码':'id'})
score = pd.DataFrame(y_pred,columns=['score'])
out_df = pd.concat([user_id,score], axis = 1)
out_df.to_csv("./output/Average.csv", index=False)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   12.6s


building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   53.3s finished


0:	learn: 434.7333049	total: 15.3ms	remaining: 7.63s
1:	learn: 305.6092813	total: 32.9ms	remaining: 8.19s
2:	learn: 215.5897786	total: 48.4ms	remaining: 8.02s
3:	learn: 152.6787757	total: 64.1ms	remaining: 7.94s
4:	learn: 108.8254783	total: 81.1ms	remaining: 8.03s
5:	learn: 78.8877400	total: 97.5ms	remaining: 8.02s
6:	learn: 58.4718333	total: 113ms	remaining: 7.94s
7:	learn: 44.9807161	total: 129ms	remaining: 7.91s
8:	learn: 35.9408229	total: 145ms	remaining: 7.88s
9:	learn: 30.2074553	total: 160ms	remaining: 7.83s
10:	learn: 26.9821118	total: 176ms	remaining: 7.81s
11:	learn: 24.9304021	total: 192ms	remaining: 7.79s
12:	learn: 23.6740782	total: 210ms	remaining: 7.88s
13:	learn: 23.0061237	total: 231ms	remaining: 8.03s
14:	learn: 22.5541429	total: 256ms	remaining: 8.29s
15:	learn: 22.2184021	total: 276ms	remaining: 8.36s
16:	learn: 21.9606549	total: 293ms	remaining: 8.32s
17:	learn: 21.7897793	total: 307ms	remaining: 8.23s
18:	learn: 21.6184158	total: 324ms	remaining: 8.2s
19:	learn: 2

165:	learn: 19.4467874	total: 2.99s	remaining: 6.02s
166:	learn: 19.4410483	total: 3.01s	remaining: 6s
167:	learn: 19.4390097	total: 3.03s	remaining: 5.99s
168:	learn: 19.4379780	total: 3.05s	remaining: 5.97s
169:	learn: 19.4361028	total: 3.07s	remaining: 5.96s
170:	learn: 19.4304232	total: 3.09s	remaining: 5.95s
171:	learn: 19.4294672	total: 3.11s	remaining: 5.92s
172:	learn: 19.4243895	total: 3.13s	remaining: 5.91s
173:	learn: 19.4229050	total: 3.14s	remaining: 5.89s
174:	learn: 19.4190133	total: 3.16s	remaining: 5.87s
175:	learn: 19.4166043	total: 3.18s	remaining: 5.85s
176:	learn: 19.4153371	total: 3.2s	remaining: 5.83s
177:	learn: 19.4144165	total: 3.22s	remaining: 5.82s
178:	learn: 19.4094160	total: 3.23s	remaining: 5.8s
179:	learn: 19.4084953	total: 3.25s	remaining: 5.78s
180:	learn: 19.4041349	total: 3.27s	remaining: 5.76s
181:	learn: 19.4001913	total: 3.28s	remaining: 5.74s
182:	learn: 19.3976459	total: 3.3s	remaining: 5.72s
183:	learn: 19.3966180	total: 3.31s	remaining: 5.69s

325:	learn: 19.1038510	total: 5.9s	remaining: 3.15s
326:	learn: 19.1025151	total: 5.92s	remaining: 3.13s
327:	learn: 19.0984956	total: 5.93s	remaining: 3.11s
328:	learn: 19.0971963	total: 5.95s	remaining: 3.09s
329:	learn: 19.0955280	total: 5.97s	remaining: 3.08s
330:	learn: 19.0951164	total: 5.99s	remaining: 3.06s
331:	learn: 19.0917139	total: 6.01s	remaining: 3.04s
332:	learn: 19.0910785	total: 6.03s	remaining: 3.02s
333:	learn: 19.0906052	total: 6.04s	remaining: 3s
334:	learn: 19.0898437	total: 6.05s	remaining: 2.98s
335:	learn: 19.0894230	total: 6.07s	remaining: 2.96s
336:	learn: 19.0886442	total: 6.09s	remaining: 2.94s
337:	learn: 19.0880269	total: 6.1s	remaining: 2.92s
338:	learn: 19.0862519	total: 6.13s	remaining: 2.91s
339:	learn: 19.0838021	total: 6.15s	remaining: 2.89s
340:	learn: 19.0822724	total: 6.17s	remaining: 2.88s
341:	learn: 19.0807413	total: 6.18s	remaining: 2.86s
342:	learn: 19.0801212	total: 6.2s	remaining: 2.84s
343:	learn: 19.0790370	total: 6.21s	remaining: 2.82s

487:	learn: 18.9252804	total: 9.13s	remaining: 224ms
488:	learn: 18.9201172	total: 9.14s	remaining: 206ms
489:	learn: 18.9192754	total: 9.16s	remaining: 187ms
490:	learn: 18.9182569	total: 9.18s	remaining: 168ms
491:	learn: 18.9164410	total: 9.21s	remaining: 150ms
492:	learn: 18.9160609	total: 9.23s	remaining: 131ms
493:	learn: 18.9157038	total: 9.24s	remaining: 112ms
494:	learn: 18.9137763	total: 9.26s	remaining: 93.5ms
495:	learn: 18.9116995	total: 9.27s	remaining: 74.8ms
496:	learn: 18.9105192	total: 9.29s	remaining: 56.1ms
497:	learn: 18.9101929	total: 9.31s	remaining: 37.4ms
498:	learn: 18.9094422	total: 9.33s	remaining: 18.7ms
499:	learn: 18.9093005	total: 9.35s	remaining: 0us


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.8s finished


### 2.Meta-model Stacking

In [92]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)

        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred

        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        #out_of_fold_predictions = np.hstack((X,out_of_fold_predictions))
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        #meta_features = np.hstack((X,meta_features))
        return self.meta_model_.predict(meta_features)

In [93]:
stacked_averaged_models = StackingAveragedModels(base_models = (Ridge,RandomForest,GBoost,Xgb,Lgbm1,Lgbm2),
                                                 meta_model = lasso)

score = score_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.7f} ({:.7f})".format(score.mean(), score.std()))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   23.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    6.3s
[Parallel(n_jobs=4)]: Done 100

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   26.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    6.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   26.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
[Parallel(n_jobs=4)]: Using backend 

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


Stacking Averaged models score: 0.0638036 (0.0004667)


In [65]:
def score(y, y_pred):
    return 1/(mean_absolute_error(y, y_pred)+1)

In [66]:
#打开文件
# path to where the data lies
dpath = './input/'
test = pd.read_csv(dpath +'test_data_org.csv',index_col='用户编码')
X_test=test.drop(['data_type'], axis=1)
X_test_index=X_test.index

In [72]:
stacked_averaged_models = StackingAveragedModels((base_models = (Ridge,RandomForest,GBoost,Xgb,Lgbm1,Lgbm2),
                                                 meta_model = lasso)
stacked_averaged_models.fit(X_trains, y_train)
stacked_pred = stacked_averaged_models.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    5.4s


building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100building tree 36 of 100

building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   23.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


...

2070:	learn: 18.5853213	total: 1m 21s	remaining: 16.9s
2071:	learn: 18.5853210	total: 1m 21s	remaining: 16.9s
2072:	learn: 18.5852942	total: 1m 21s	remaining: 16.8s
2073:	learn: 18.5852536	total: 1m 21s	remaining: 16.8s
2074:	learn: 18.5844086	total: 1m 21s	remaining: 16.7s
2075:	learn: 18.5841387	total: 1m 21s	remaining: 16.7s
2076:	learn: 18.5841373	total: 1m 21s	remaining: 16.7s
2077:	learn: 18.5835453	total: 1m 21s	remaining: 16.6s
2078:	learn: 18.5832034	total: 1m 21s	remaining: 16.6s
2079:	learn: 18.5831896	total: 1m 21s	remaining: 16.5s
2080:	learn: 18.5826867	total: 1m 21s	remaining: 16.5s
2081:	learn: 18.5826711	total: 1m 21s	remaining: 16.5s
2082:	learn: 18.5823222	total: 1m 22s	remaining: 16.4s
2083:	learn: 18.5822962	total: 1m 22s	remaining: 16.4s
2084:	learn: 18.5822628	total: 1m 22s	remaining: 16.3s
2085:	learn: 18.5821128	total: 1m 22s	remaining: 16.3s
2086:	learn: 18.5821128	total: 1m 22s	remaining: 16.3s
2087:	learn: 18.5818534	total: 1m 22s	remaining: 16.2s
2088:	lear

2225:	learn: 18.5563564	total: 1m 27s	remaining: 10.8s
2226:	learn: 18.5555758	total: 1m 27s	remaining: 10.7s
2227:	learn: 18.5554154	total: 1m 27s	remaining: 10.7s
2228:	learn: 18.5553949	total: 1m 27s	remaining: 10.6s
2229:	learn: 18.5550294	total: 1m 27s	remaining: 10.6s
2230:	learn: 18.5550294	total: 1m 27s	remaining: 10.6s
2231:	learn: 18.5550016	total: 1m 27s	remaining: 10.5s
2232:	learn: 18.5549669	total: 1m 27s	remaining: 10.5s
2233:	learn: 18.5549341	total: 1m 27s	remaining: 10.4s
2234:	learn: 18.5549034	total: 1m 27s	remaining: 10.4s
2235:	learn: 18.5545055	total: 1m 27s	remaining: 10.4s
2236:	learn: 18.5543809	total: 1m 27s	remaining: 10.3s
2237:	learn: 18.5543306	total: 1m 27s	remaining: 10.3s
2238:	learn: 18.5542362	total: 1m 27s	remaining: 10.3s
2239:	learn: 18.5542361	total: 1m 27s	remaining: 10.2s
2240:	learn: 18.5531892	total: 1m 28s	remaining: 10.2s
2241:	learn: 18.5531790	total: 1m 28s	remaining: 10.1s
2242:	learn: 18.5529106	total: 1m 28s	remaining: 10.1s
2243:	lear

2380:	learn: 18.5369734	total: 1m 33s	remaining: 4.7s
2381:	learn: 18.5369707	total: 1m 33s	remaining: 4.66s
2382:	learn: 18.5367617	total: 1m 34s	remaining: 4.62s
2383:	learn: 18.5366535	total: 1m 34s	remaining: 4.58s
2384:	learn: 18.5366404	total: 1m 34s	remaining: 4.54s
2385:	learn: 18.5349273	total: 1m 34s	remaining: 4.5s
2386:	learn: 18.5349085	total: 1m 34s	remaining: 4.46s
2387:	learn: 18.5349060	total: 1m 34s	remaining: 4.42s
2388:	learn: 18.5349020	total: 1m 34s	remaining: 4.38s
2389:	learn: 18.5348994	total: 1m 34s	remaining: 4.34s
2390:	learn: 18.5348967	total: 1m 34s	remaining: 4.3s
2391:	learn: 18.5344825	total: 1m 34s	remaining: 4.26s
2392:	learn: 18.5344687	total: 1m 34s	remaining: 4.22s
2393:	learn: 18.5344452	total: 1m 34s	remaining: 4.18s
2394:	learn: 18.5340533	total: 1m 34s	remaining: 4.14s
2395:	learn: 18.5340509	total: 1m 34s	remaining: 4.11s
2396:	learn: 18.5340415	total: 1m 34s	remaining: 4.07s
2397:	learn: 18.5331007	total: 1m 34s	remaining: 4.03s
2398:	learn: 

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_job

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36'] ['用户实名制是否通过核实', '用户年龄', '是否大学生客户', '是否黑名单客户', '是否4G不健康客户', '用户网龄（月）', '用户最近一次缴费距今时长（月）', '缴费用户最近一次缴费金额（元）', '用户近6个月平均消费值（元）', '用户账单当月总费用（元）', '用户当月账户余额（元）', '缴费用户当前是否欠费缴费', '当月通话交往圈人数', '是否经常逛商场的人', '近三个月月均商场出现次数', '当月是否逛过福州仓山万达', '当月是否到过福州山姆会员店', '当月是否看电影', '当月是否景点游览', '当月是否体育场馆消费', '当月网购类应用使用次数', '当月物流快递类应用使用次数', '当月金融理财类应用使用总次数', '当月视频播放类应用使用次数', '当月飞机类应用使用次数', '当月火车类应用使用次数', '当月旅游资讯类应用使用次数', 'data_type', '充值途径', '话费稳定性', '资金稳定性', '用户话费敏感度_0', '用户话费敏感度_1', '用户话费敏感度_2', '用户话费敏感度_3', '用户话费敏感度_4', '用户话费敏感度_5']
expected f12, f28, f18, f14, f4, f29, f0, f22, f6, f1, f9, f36, f2, f20, f32, f5, f31, f26, f35, f33, f17, f27, f19, f11, f23, f7, f24, f13, f15, f34, f3, f25, f8, f10, f30, f16, f21 in input data
t

In [78]:
# XGBoost
Xgb.fit(train, y_train)
xgb_train_pred = Xgb.predict(train)
xgb_pred = Xgb.predict(test)
print(score(y_train, xgb_train_pred))

0.06639449333409013


In [79]:
# LightGBM1
Lgbm1.fit(train, y_train)
lgb_train_pred1 = Lgbm1.predict(train)
lgb_pred1 = Lgbm1.predict(test.values)
print(score(y_train, lgb_train_pred1))

0.0710517345539274


In [80]:
# LightGBM2
Lgbm2.fit(train, y_train)
lgb_train_pred2 = Lgbm2.predict(train)
lgb_pred2 = Lgbm2.predict(test.values)
print(score(y_train, lgb_train_pred2))

0.071477747744838


In [81]:
# CatBoostRegressor
Catb.fit(train, y_train)
cat_train_pred = Catb.predict(train)
cat_pred = Catb.predict(test.values)
print(score(y_train, cat_pred))

0:	learn: 527.0494887	total: 22.9ms	remaining: 57.3s
1:	learn: 448.5491962	total: 45.8ms	remaining: 57.2s
2:	learn: 381.9119196	total: 67.9ms	remaining: 56.5s
3:	learn: 325.2635959	total: 91.2ms	remaining: 56.9s
4:	learn: 277.2989811	total: 114ms	remaining: 57s
5:	learn: 236.6582625	total: 138ms	remaining: 57.3s
6:	learn: 202.1626458	total: 160ms	remaining: 57s
7:	learn: 172.9319162	total: 184ms	remaining: 57.3s
8:	learn: 147.7830739	total: 207ms	remaining: 57.4s
9:	learn: 126.4218230	total: 245ms	remaining: 1m 1s
10:	learn: 108.4621355	total: 280ms	remaining: 1m 3s
11:	learn: 93.2531046	total: 310ms	remaining: 1m 4s
12:	learn: 80.5220426	total: 348ms	remaining: 1m 6s
13:	learn: 69.9190583	total: 387ms	remaining: 1m 8s
14:	learn: 60.9651669	total: 412ms	remaining: 1m 8s
15:	learn: 53.4304096	total: 439ms	remaining: 1m 8s
16:	learn: 47.0479008	total: 469ms	remaining: 1m 8s
17:	learn: 41.9112085	total: 507ms	remaining: 1m 9s
18:	learn: 37.6686582	total: 543ms	remaining: 1m 10s
19:	learn:

164:	learn: 19.4216008	total: 4.69s	remaining: 1m 6s
165:	learn: 19.4186425	total: 4.72s	remaining: 1m 6s
166:	learn: 19.4141167	total: 4.75s	remaining: 1m 6s
167:	learn: 19.4095666	total: 4.78s	remaining: 1m 6s
168:	learn: 19.4046812	total: 4.8s	remaining: 1m 6s
169:	learn: 19.3999955	total: 4.83s	remaining: 1m 6s
170:	learn: 19.3982386	total: 4.85s	remaining: 1m 6s
171:	learn: 19.3959157	total: 4.88s	remaining: 1m 5s
172:	learn: 19.3941343	total: 4.91s	remaining: 1m 6s
173:	learn: 19.3916678	total: 4.93s	remaining: 1m 5s
174:	learn: 19.3878239	total: 4.96s	remaining: 1m 5s
175:	learn: 19.3821483	total: 4.98s	remaining: 1m 5s
176:	learn: 19.3803496	total: 5s	remaining: 1m 5s
177:	learn: 19.3778298	total: 5.03s	remaining: 1m 5s
178:	learn: 19.3752412	total: 5.05s	remaining: 1m 5s
179:	learn: 19.3724050	total: 5.08s	remaining: 1m 5s
180:	learn: 19.3694164	total: 5.11s	remaining: 1m 5s
181:	learn: 19.3670988	total: 5.13s	remaining: 1m 5s
182:	learn: 19.3626704	total: 5.16s	remaining: 1m 

325:	learn: 19.0670247	total: 8.77s	remaining: 58.5s
326:	learn: 19.0661185	total: 8.8s	remaining: 58.5s
327:	learn: 19.0645807	total: 8.83s	remaining: 58.5s
328:	learn: 19.0635189	total: 8.85s	remaining: 58.4s
329:	learn: 19.0615911	total: 8.88s	remaining: 58.4s
330:	learn: 19.0610430	total: 8.9s	remaining: 58.3s
331:	learn: 19.0577392	total: 8.93s	remaining: 58.3s
332:	learn: 19.0567190	total: 8.95s	remaining: 58.2s
333:	learn: 19.0558073	total: 8.97s	remaining: 58.2s
334:	learn: 19.0539457	total: 9s	remaining: 58.2s
335:	learn: 19.0515054	total: 9.03s	remaining: 58.1s
336:	learn: 19.0496202	total: 9.05s	remaining: 58.1s
337:	learn: 19.0491752	total: 9.07s	remaining: 58s
338:	learn: 19.0476385	total: 9.1s	remaining: 58s
339:	learn: 19.0458411	total: 9.12s	remaining: 57.9s
340:	learn: 19.0449652	total: 9.14s	remaining: 57.9s
341:	learn: 19.0444197	total: 9.17s	remaining: 57.9s
342:	learn: 19.0436294	total: 9.2s	remaining: 57.8s
343:	learn: 19.0414847	total: 9.22s	remaining: 57.8s
344:

485:	learn: 18.8698394	total: 12.8s	remaining: 53.2s
486:	learn: 18.8689547	total: 12.9s	remaining: 53.2s
487:	learn: 18.8676157	total: 12.9s	remaining: 53.1s
488:	learn: 18.8667394	total: 12.9s	remaining: 53.1s
489:	learn: 18.8658709	total: 12.9s	remaining: 53.1s
490:	learn: 18.8649688	total: 13s	remaining: 53.1s
491:	learn: 18.8639541	total: 13s	remaining: 53s
492:	learn: 18.8629667	total: 13s	remaining: 53s
493:	learn: 18.8615870	total: 13s	remaining: 53s
494:	learn: 18.8606492	total: 13.1s	remaining: 52.9s
495:	learn: 18.8601475	total: 13.1s	remaining: 52.9s
496:	learn: 18.8593148	total: 13.1s	remaining: 52.9s
497:	learn: 18.8588375	total: 13.1s	remaining: 52.8s
498:	learn: 18.8570272	total: 13.2s	remaining: 52.8s
499:	learn: 18.8556401	total: 13.2s	remaining: 52.7s
500:	learn: 18.8545086	total: 13.2s	remaining: 52.7s
501:	learn: 18.8537722	total: 13.2s	remaining: 52.7s
502:	learn: 18.8534849	total: 13.3s	remaining: 52.7s
503:	learn: 18.8522725	total: 13.3s	remaining: 52.6s
504:	le

646:	learn: 18.7274084	total: 16.9s	remaining: 48.5s
647:	learn: 18.7256232	total: 17s	remaining: 48.5s
648:	learn: 18.7249182	total: 17s	remaining: 48.5s
649:	learn: 18.7242203	total: 17s	remaining: 48.5s
650:	learn: 18.7231336	total: 17s	remaining: 48.4s
651:	learn: 18.7225708	total: 17.1s	remaining: 48.4s
652:	learn: 18.7219041	total: 17.1s	remaining: 48.3s
653:	learn: 18.7208556	total: 17.1s	remaining: 48.3s
654:	learn: 18.7200400	total: 17.1s	remaining: 48.3s
655:	learn: 18.7190243	total: 17.2s	remaining: 48.3s
656:	learn: 18.7179920	total: 17.2s	remaining: 48.3s
657:	learn: 18.7173338	total: 17.2s	remaining: 48.2s
658:	learn: 18.7161857	total: 17.3s	remaining: 48.2s
659:	learn: 18.7156842	total: 17.3s	remaining: 48.2s
660:	learn: 18.7149776	total: 17.3s	remaining: 48.2s
661:	learn: 18.7142599	total: 17.3s	remaining: 48.1s
662:	learn: 18.7132120	total: 17.4s	remaining: 48.1s
663:	learn: 18.7123129	total: 17.4s	remaining: 48.1s
664:	learn: 18.7113630	total: 17.4s	remaining: 48.1s
6

...

1739:	learn: 18.1245653	total: 46.3s	remaining: 20.2s
1740:	learn: 18.1242556	total: 46.3s	remaining: 20.2s
1741:	learn: 18.1238485	total: 46.4s	remaining: 20.2s
1742:	learn: 18.1235755	total: 46.4s	remaining: 20.1s
1743:	learn: 18.1231122	total: 46.4s	remaining: 20.1s
1744:	learn: 18.1227959	total: 46.4s	remaining: 20.1s
1745:	learn: 18.1223956	total: 46.5s	remaining: 20.1s
1746:	learn: 18.1218854	total: 46.5s	remaining: 20s
1747:	learn: 18.1211282	total: 46.5s	remaining: 20s
1748:	learn: 18.1209717	total: 46.5s	remaining: 20s
1749:	learn: 18.1203983	total: 46.6s	remaining: 20s
1750:	learn: 18.1198741	total: 46.6s	remaining: 19.9s
1751:	learn: 18.1197687	total: 46.6s	remaining: 19.9s
1752:	learn: 18.1192233	total: 46.6s	remaining: 19.9s
1753:	learn: 18.1189459	total: 46.7s	remaining: 19.8s
1754:	learn: 18.1185567	total: 46.7s	remaining: 19.8s
1755:	learn: 18.1184160	total: 46.7s	remaining: 19.8s
1756:	learn: 18.1182439	total: 46.8s	remaining: 19.8s
1757:	learn: 18.1179148	total: 46.8s

1894:	learn: 18.0604827	total: 50.2s	remaining: 16s
1895:	learn: 18.0599141	total: 50.3s	remaining: 16s
1896:	learn: 18.0596639	total: 50.3s	remaining: 16s
1897:	learn: 18.0594182	total: 50.3s	remaining: 16s
1898:	learn: 18.0592297	total: 50.3s	remaining: 15.9s
1899:	learn: 18.0586586	total: 50.4s	remaining: 15.9s
1900:	learn: 18.0578352	total: 50.4s	remaining: 15.9s
1901:	learn: 18.0576819	total: 50.4s	remaining: 15.8s
1902:	learn: 18.0575197	total: 50.4s	remaining: 15.8s
1903:	learn: 18.0573434	total: 50.5s	remaining: 15.8s
1904:	learn: 18.0571692	total: 50.5s	remaining: 15.8s
1905:	learn: 18.0566945	total: 50.5s	remaining: 15.7s
1906:	learn: 18.0565749	total: 50.6s	remaining: 15.7s
1907:	learn: 18.0565095	total: 50.6s	remaining: 15.7s
1908:	learn: 18.0560826	total: 50.6s	remaining: 15.7s
1909:	learn: 18.0556532	total: 50.6s	remaining: 15.6s
1910:	learn: 18.0554669	total: 50.7s	remaining: 15.6s
1911:	learn: 18.0552105	total: 50.7s	remaining: 15.6s
1912:	learn: 18.0549350	total: 50.7s

2051:	learn: 18.0015604	total: 54.2s	remaining: 11.8s
2052:	learn: 18.0007716	total: 54.3s	remaining: 11.8s
2053:	learn: 18.0001818	total: 54.3s	remaining: 11.8s
2054:	learn: 17.9997969	total: 54.3s	remaining: 11.8s
2055:	learn: 17.9992329	total: 54.3s	remaining: 11.7s
2056:	learn: 17.9988118	total: 54.4s	remaining: 11.7s
2057:	learn: 17.9986401	total: 54.4s	remaining: 11.7s
2058:	learn: 17.9981501	total: 54.4s	remaining: 11.7s
2059:	learn: 17.9975990	total: 54.4s	remaining: 11.6s
2060:	learn: 17.9969241	total: 54.4s	remaining: 11.6s
2061:	learn: 17.9962656	total: 54.5s	remaining: 11.6s
2062:	learn: 17.9959641	total: 54.5s	remaining: 11.5s
2063:	learn: 17.9955997	total: 54.5s	remaining: 11.5s
2064:	learn: 17.9953914	total: 54.5s	remaining: 11.5s
2065:	learn: 17.9945917	total: 54.6s	remaining: 11.5s
2066:	learn: 17.9941305	total: 54.6s	remaining: 11.4s
2067:	learn: 17.9940059	total: 54.6s	remaining: 11.4s
2068:	learn: 17.9933852	total: 54.6s	remaining: 11.4s
2069:	learn: 17.9931861	tota

2208:	learn: 17.9384804	total: 58.2s	remaining: 7.67s
2209:	learn: 17.9382248	total: 58.2s	remaining: 7.64s
2210:	learn: 17.9380664	total: 58.3s	remaining: 7.62s
2211:	learn: 17.9373350	total: 58.3s	remaining: 7.59s
2212:	learn: 17.9368468	total: 58.3s	remaining: 7.56s
2213:	learn: 17.9367785	total: 58.3s	remaining: 7.54s
2214:	learn: 17.9364967	total: 58.4s	remaining: 7.51s
2215:	learn: 17.9358179	total: 58.4s	remaining: 7.48s
2216:	learn: 17.9354583	total: 58.4s	remaining: 7.46s
2217:	learn: 17.9350592	total: 58.4s	remaining: 7.43s
2218:	learn: 17.9347724	total: 58.5s	remaining: 7.4s
2219:	learn: 17.9343671	total: 58.5s	remaining: 7.38s
2220:	learn: 17.9337315	total: 58.5s	remaining: 7.35s
2221:	learn: 17.9335323	total: 58.5s	remaining: 7.32s
2222:	learn: 17.9328110	total: 58.6s	remaining: 7.3s
2223:	learn: 17.9320800	total: 58.6s	remaining: 7.27s
2224:	learn: 17.9317272	total: 58.6s	remaining: 7.24s
2225:	learn: 17.9313483	total: 58.6s	remaining: 7.22s
2226:	learn: 17.9309401	total:

2363:	learn: 17.8795479	total: 1m 2s	remaining: 3.57s
2364:	learn: 17.8788661	total: 1m 2s	remaining: 3.55s
2365:	learn: 17.8784115	total: 1m 2s	remaining: 3.52s
2366:	learn: 17.8781370	total: 1m 2s	remaining: 3.49s
2367:	learn: 17.8777597	total: 1m 2s	remaining: 3.47s
2368:	learn: 17.8773922	total: 1m 2s	remaining: 3.44s
2369:	learn: 17.8770990	total: 1m 2s	remaining: 3.42s
2370:	learn: 17.8762083	total: 1m 2s	remaining: 3.39s
2371:	learn: 17.8756744	total: 1m 2s	remaining: 3.36s
2372:	learn: 17.8751880	total: 1m 2s	remaining: 3.34s
2373:	learn: 17.8747158	total: 1m 2s	remaining: 3.31s
2374:	learn: 17.8736444	total: 1m 2s	remaining: 3.28s
2375:	learn: 17.8734108	total: 1m 2s	remaining: 3.26s
2376:	learn: 17.8730240	total: 1m 2s	remaining: 3.23s
2377:	learn: 17.8726741	total: 1m 2s	remaining: 3.21s
2378:	learn: 17.8723745	total: 1m 2s	remaining: 3.18s
2379:	learn: 17.8718360	total: 1m 2s	remaining: 3.15s
2380:	learn: 17.8716223	total: 1m 2s	remaining: 3.13s
2381:	learn: 17.8713723	tota

In [ ]:
'''MAE on the entire Train data when averaging'''

print('MAE score on train data:')
print(score(y_train,stacked_train_pred*0.7 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

In [82]:
# Ensemble prediction
ensemble = stacked_pred*0.7 + xgb_train_pred*0.1 + lgb_train_pred1*0.1+ lgb_train_pred2*0.1

In [85]:
y_pred = stacked_pred
#四舍五入，取整
for i in range(50000):
    y_pred[i]=round(y_pred[i])
y_pred=y_pred.astype(int)  

In [86]:
#生成提交结果
user_id=pd.DataFrame(X_test_index).rename(columns = {'用户编码':'id'})
score = pd.DataFrame(y_pred,columns=['score'])
out_df = pd.concat([user_id,score], axis = 1)
out_df.to_csv("./output/Metastaking.csv", index=False)

In [87]:
# scikit-learn k-fold cross-validation
from numpy import array
from sklearn.model_selection import KFold
# data sample
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# prepare cross validation
kfold = KFold(n_splits=3, shuffle = True, random_state= 1)
# enumerate splits
for train, test in kfold.split(data):
    print('train: %s, test: %s' % (data[train], data[test]))

train: [0.1 0.4 0.5 0.6], test: [0.2 0.3]
train: [0.2 0.3 0.4 0.6], test: [0.1 0.5]
train: [0.1 0.2 0.3 0.5], test: [0.4 0.6]
